### Stats about text to tokenise

In [5]:
with open("data/the-verdict.txt", "r", encoding="utf-8") as book_file:
    raw_text = book_file.read()

print("Total number of characters:", len(raw_text))

Total number of characters: 20479


In [9]:
print("First 100 characters:\n", raw_text[:99])

First 100 characters:
 I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 
